In [ ]:
# 네이버 뉴스 크롤링 -> 8/29 8/31 9/1 실행

from selenium import webdriver
import pandas as pd
import numpy as np


url = 'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1='


category = ['100#&date=%2000:00:00&page=',
            '101#&date=%2000:00:00&page=',
            '102#&date=%2000:00:00&page=',
            '103#&date=%2000:00:00&page=',
            '104#&date=%2000:00:00&page=',
            '105#&date=%2000:00:00&page=']
#정치, 경제, 사회, 생활/문화, 세계, IT/과학
driver = webdriver.Chrome('chromedriver')
driver.implicitly_wait(1)
driver.get('https://naver.com')

f = pd.DataFrame()

for cate in category:
    
    for page in range(1, 301):
        
        home = url + cate + str(page)

        driver.get(home)

        a = driver.find_element_by_css_selector('#section_body').text.split('\n')
        
        b = []
        i = 0
        while i < len(a):
            if (a[i].replace(',','').isdigit()) | (a[i] == '이상') | (a[i] == '+'):
                pass
            else:
                b.append(a[i])

            i += 1

        c = []
        
        for j in b:
            if j != '동영상기사':
                c.append(j)       
        
        d = []
        if len(c) == 60:

            for k in range(len(c)):
                if k % 3 == 0:
                    d.append(c[k].replace('\'', '').replace('"','').replace('`','').replace('“','').replace('”',''))
                elif k % 3 == 1:
                    pass
                else:
                    d.append(c[k][:4])

            e = pd.DataFrame(np.array(d).reshape(-1,2))
            e['category'] = category.index(cate)


            f = pd.concat([f, e])
        else:
            print('pass '+str(category.index(cate)) + ' ' + str(page))

In [ ]:
f.columns = ['title','신문사','category']

In [ ]:
g = f.loc[f['신문사'] != '연합뉴스'].reset_index(drop=True)
h = g.loc[~(g['신문사'].str.startswith('YTN'))]
#정치, 경제, 사회, 생활/문화, 세계, IT/과학
h['topic_idx'] = np.where(h['category'] == 0, 6,
                         np.where(h['category'] == 1, 1,
                                 np.where(h['category'] == 2, 2,
                                         np.where(h['category'] == 3, 3,
                                                 np.where(h['category'] == 4, 4,
                                                         0)))))

h['topic_idx'].value_counts()

In [ ]:
h.to_csv('0901_300_1.csv', index = None)